In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


In [2]:
tss_annotation = doc['tss_annotation']

In [3]:
from collections import defaultdict
import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 13)


In [4]:
meta_samples = pd.read_csv('../raw_metaSamples.tsv',sep="\t",index_col=0)
meta_samples.head()

,Tissue,Experiment,Name,Full
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC1,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC2,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,BMDM1hKLA,GRO,BMDM1hKLA_GRO1,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,BMDM1hKLA,GROCap,BMDM1hKLA_GROCap1,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...
CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_R1_001.fastq.gz,BMDMwt,ATAC,BMDMwt_ATAC1,CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_...


## Load anno_peaks. 
Later will filter based on Distance to TSS

## Load peaks_tissues matrix
Later will be used to merge two data types in a tissue and for more confident peaks if they have more than 1 tissues present

## Load tss

## Load txn_df




In [5]:
anno_peaks = pd.read_csv('Results/merged/peaks_with_tss_distances.tsv',sep='\t',index_col=0)
anno_peaks.head()

,Chr,Start,End,Strand,Stat,Annotation,Nearest TSS,Distance to TSS,isSameStrand,Nearest gene
ID,,,,,,,,,,
0,NW_020822452.1,16590187,16590584,+,16.227333,promoter-TSS (ID=id181373;Parent=rna15526;Dbxr...,XM_027399668.1,-462.0,False,Clic4
1,NW_020822597.1,23595304,23595489,+,5.740000,Intergenic,XM_027420351.1,-23850.0,True,Baz2b
2,NW_020822680.1,728607,728829,+,7.579579,promoter-TSS (ID=id609942;Parent=rna53731;Dbxr...,XM_027432478.1,746.0,False,Irak1
3,NW_020822602.1,10883074,10883337,+,3.992000,Intergenic,XM_027423054.1,9079.0,True,Gbgt1
4,NW_020822601.1,37427816,37427999,+,129.031738,promoter-TSS (ID=id474643;Parent=rna41513;Dbxr...,XM_027421990.1,-11.0,True,Shf


In [6]:
peaks_tissue = pd.read_csv("Results/merged/samples.merge.peaksexpression",index_col=0,sep="\t")
peaks_tissue.head()

,f04_peaks/MiscOrgans_START1_and_MiscOrgans_startinput1,f04_peaks/Liver_START2_and_Liver_startinput1,f04_peaks/Brain_GROCap1_and_Brain_GRO1,f04_peaks/Liver_START1_and_Liver_startinput1,f04_peaks/Lung_GROCap1_and_Lung_GRO1,f04_peaks/Brain_START3_and_Brain_startinput1,f04_peaks/Heart_START1_and_Heart_startinput1,f04_peaks/CHO_GROCap2_and_CHO_GRO1,f04_peaks/Heart_START2_and_Heart_startinput1,f04_peaks/Liver_GROCap1_and_Liver_GRO1,...,f04_peaks/MiscOrgans_START2_and_MiscOrgans_startinput1,f04_peaks/Spleen_START1_and_Spleen_startinput1,f04_peaks/Lung_START2_and_Lung_startinput1,f04_peaks/CHO_START1_and_CHO_startinput1,f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_startinput1,f04_peaks/Kidney_START2_and_Kidney_startinput1,f04_peaks/Kidney_GROCap1_and_Kidney_GRO1,f04_peaks/CHO_GROCap1_and_CHO_GRO1,f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1,f04_peaks/Spleen_START2_and_Spleen_startinput1
0,1.41,0.0,1.270,0.0,6.54,0.00,1.67,31.8,0.0,0.0,...,0.00,0.00,0.0,5.38,0.00,0.0,6.39,33.00,0.00,0.00
1,1.97,0.0,2.060,3.3,0.00,0.00,0.00,0.0,0.0,0.0,...,5.98,1.05,40.6,0.00,5.69,0.0,0.00,0.00,0.00,5.72
2,1.41,0.0,0.952,0.0,3.79,0.00,0.00,22.6,0.0,0.0,...,0.00,0.00,0.0,6.82,1.73,0.0,1.83,21.60,5.66,2.51
3,3.95,0.0,0.000,0.0,0.00,3.24,1.67,0.0,0.0,0.0,...,2.10,3.46,0.0,3.14,0.00,0.0,0.00,2.63,2.45,2.29
4,165.30,0.0,1.860,81.8,0.00,43.50,51.20,0.0,0.0,7.3,...,58.80,18.20,0.0,91.90,0.00,0.0,1.52,4.78,3.39,158.90


#### Read in tss and txn_df

In [7]:
tss = pd.read_csv(tss_annotation,sep='\t',index_col=0)

tss.head()

,Chr,Start,End,Strand,ID,Parent,Dbxref,Name,gbkey,gene,...,product,transcript_id,Note,exception,end_range,partial,start_range,inference,Length,actual_start
"ID=rna7;Parent=gene4;Dbxref=GeneID:100754303,Genbank:XM_027386529.1;Name=XM_027386529.1;gbkey=mRNA;gene=Bmp4;model_evidence=Supporting evidence includes similarity to: 29 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 15 samples with support for all annotated introns;product=bone morphogenetic protein 4%2C transcript variant X3;transcript_id=XM_027386529.1",NW_020822366.1,717871,722308,-,rna7,gene4,"GeneID:100754303,Genbank:XM_027386529.1",XM_027386529.1,mRNA,Bmp4,...,bone morphogenetic protein 4%2C transcript var...,XM_027386529.1,NaN,NaN,NaN,NaN,NaN,NaN,4437,722308
"ID=rna6;Parent=gene4;Dbxref=GeneID:100754303,Genbank:XM_027386528.1;Name=XM_027386528.1;gbkey=mRNA;gene=Bmp4;model_evidence=Supporting evidence includes similarity to: 29 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 18 samples with support for all annotated introns;product=bone morphogenetic protein 4%2C transcript variant X2;transcript_id=XM_027386528.1",NW_020822366.1,717871,724610,-,rna6,gene4,"GeneID:100754303,Genbank:XM_027386528.1",XM_027386528.1,mRNA,Bmp4,...,bone morphogenetic protein 4%2C transcript var...,XM_027386528.1,NaN,NaN,NaN,NaN,NaN,NaN,6739,724610
"ID=rna10;Parent=gene6;Dbxref=GeneID:100752216,Genbank:XM_027386531.1;Name=XM_027386531.1;gbkey=mRNA;gene=Cdkn3;model_evidence=Supporting evidence includes similarity to: 5 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 98 samples with support for all annotated introns;product=cyclin dependent kinase inhibitor 3%2C transcript variant X1;transcript_id=XM_027386531.1",NW_020822366.1,1085435,1097417,+,rna10,gene6,"GeneID:100752216,Genbank:XM_027386531.1",XM_027386531.1,mRNA,Cdkn3,...,cyclin dependent kinase inhibitor 3%2C transcr...,XM_027386531.1,NaN,NaN,NaN,NaN,NaN,NaN,11982,1085435
"ID=rna12;Parent=gene6;Dbxref=GeneID:100752216,Genbank:XM_027386533.1;Name=XM_027386533.1;gbkey=mRNA;gene=Cdkn3;model_evidence=Supporting evidence includes similarity to: 1 Protein%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 84 samples with support for all annotated introns;product=cyclin dependent kinase inhibitor 3%2C transcript variant X3;transcript_id=XM_027386533.1",NW_020822366.1,1085744,1097417,+,rna12,gene6,"GeneID:100752216,Genbank:XM_027386533.1",XM_027386533.1,mRNA,Cdkn3,...,cyclin dependent kinase inhibitor 3%2C transcr...,XM_027386533.1,NaN,NaN,NaN,NaN,NaN,NaN,11673,1085744
"ID=rna13;Parent=gene7;Dbxref=GeneID:103160710,Genbank:XM_027386534.1;Name=XM_027386534.1;gbkey=mRNA;gene=Cnih1;model_evidence=Supporting evidence includes similarity to: 10 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 99 samples with support for all annotated introns;product=cornichon family AMPA receptor auxiliary protein 1%2C transcript variant X2;transcript_id=XM_027386534.1",NW_020822366.1,1101897,1114413,-,rna13,gene7,"GeneID:103160710,Genbank:XM_027386534.1",XM_027386534.1,mRNA,Cnih1,...,cornichon family AMPA receptor auxiliary prote...,XM_027386534.1,NaN,NaN,NaN,NaN,NaN,NaN,12516,1114413


In [8]:
txn_df = pickle.load(open('Results/tss_annotation/txn_df_02.p','rb'))# pd.read_csv('Results/txn_df.csv',index_col=0)
txn_df.head()

,peaks,closest_to_X_numberOfPeaks,bin_-1000_1000,sameStrand_bin_-1000_1000,samples,minDistance,minDistancePeak,maxValue,maxPeakId,maxSample,Chr,Start,End,Strand,gene,transcript,maxSamples,hasGene,Number of SS
XM_027386529.1,"[1394, 3038, 17726, 21888, 24414, 35067, 69088...",101,"[90264, 97971, 124234, 181215, 317724, 348823]","[90264, 181215, 348823]","[MiscOrgans_START1_and_MiscOrgans_startinput1,...",63.0,348823,59.2,348823,NaN,NW_020822366.1,722245,722246,-,Bmp4,XM_027386529.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,3
XM_027386528.1,"[17698, 163408, 224556, 226028, 268860, 275685...",28,"[163408, 308710]",[308710],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",27.0,308710,11864.6,308710,NaN,NW_020822366.1,724583,724584,-,Bmp4,XM_027386528.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1
XM_027386531.1,"[17679, 17707, 83014, 89129, 99005, 108459, 10...",29,"[138244, 237253]",[237253],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",74.0,237253,15.3,237253,NaN,NW_020822366.1,1085509,1085510,+,Cdkn3,XM_027386531.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1
XM_027386533.1,"[68904, 137124, 249586, 270147, 335001, 400304...",7,[],[],[],inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],False,0
XM_027386534.1,"[14157, 68988, 69200, 116089, 132667, 167473, ...",17,"[132667, 298307, 350581, 353645]",[298307],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",88.0,298307,57.2,298307,NaN,NW_020822366.1,1114325,1114326,-,Cnih1,XM_027386534.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1


### Merge tissue columns

In [9]:
def merge_two_columns(df, mapping_dict):
    '''Function that merges columns by taking the mean of them.
    Merges based on if they have the same element in the meta_samples meta_column.
    Returns:
        new_df: Dataframe but with the columns of interest merged. Also column names are now based
                on the unique meta_samples[meta_column].
    '''

    vals = mapping_dict.keys()#np.unique(meta_samples[meta_column].values)
    new_df = pd.DataFrame(index=df.index, columns=vals)
    for i in vals:
        if not mapping_dict[i] == []:#(meta_samples[meta_column] == i).any():
            new_col = (df.loc[:, mapping_dict[i]])
            new_col = new_col.mean(axis=1)
            new_df.loc[:, i] = new_col
    new_df = new_df.loc[:, ~(new_df.isnull().all())]
    return new_df

In [10]:
tissue_to_peaks = defaultdict(list)
for val in peaks_tissue.columns.values:
    count = 0
    for t in np.unique(meta_samples["Tissue"].values):
        if t in val:
            count += 1
            tissue_to_peaks[t].append(val)
            if count > 1:
                print("This sample has multiple tissues associated with it")
                print("Sample: ",val)

In [11]:
peaks_tissue_merged = merge_two_columns(peaks_tissue,mapping_dict=tissue_to_peaks)

In [12]:
peaks_tissue_merged.head()

,Heart,Spleen,MiscOrgans,FemaleReproductive,CHO,BMDM1hKLA,Brain,Lung,Liver,BMDMwt,Muscle,Kidney,Pancreas
0,0.835,0.000,0.705,0.630,26.996,84.2,0.790000,2.180000,0.0,0.00,0.715,2.130000,0.000
1,0.000,3.385,3.975,4.265,0.000,0.0,1.608571,16.596667,1.1,0.00,0.545,1.050000,0.000
2,0.000,1.255,0.705,4.025,19.044,0.0,1.887429,2.386667,0.0,5.66,1.100,1.870000,1.490
3,0.835,2.875,3.025,6.000,1.680,0.0,3.914286,1.633333,0.0,2.45,1.495,1.153333,2.385
4,25.600,88.550,112.050,136.550,20.292,0.0,189.308571,50.966667,29.7,3.39,124.150,43.640000,95.500


### Create directory of different peak groups

In [13]:
if not os.path.exists("Results/peak_groups"):
    os.mkdir("Results/peak_groups")

In [14]:
def save_peak_as_bed(peaks_df,f_save, name='index',score_col = 'Peak Score',is_sort = True):
    ## Assumes there is Chr,Start,End,Name is index
    ## Peak file is 1-index, bed files are 0, so need to shift everything by one
    
    if name == 'index':
        peaks_df.loc[:,'Name'] = peaks_df.index
    else:
        peaks_df.loc[:,'Name'] = peaks_df[name]
    
    if score_col == '': #If no scoring col, set to 0
        peaks_df.loc[:,'Score'] = 0
        score_col = 'Score'
        
    peaks_df.loc[:,'Start'] = peaks_df['Start'] - 1
    peaks_df.loc[:,'End'] = peaks_df['End'] - 1
    
    peaks_df = peaks_df[['Chr','Start','End','Name',score_col,'Strand']]
    
    if is_sort:
        peaks_df.sort_values(['Chr','Start'],inplace=True)
        
    if not f_save.endswith('.bed'):
        f_save = f_save + '.bed'
    peaks_df.to_csv(f_save ,sep='\t',index=False,header=None)
    return
    

## a) Separate confident promoters and confident ncRNA 
Define by distance to RefSeq being 0 and ncRNA far >40kb from a gene and in at least two tissues

### c) Predicted promoters vs RefSeq promoters

### d) Promoters organized based on distance to refseq (so heatmap is not sorted by heat but by distance)

### e) Only look at peaks specific for each tissue

In [15]:
def filter_only_with_divergent(df,divergent_threshold = 500 ):
    inds_to_keep = []
    for ind,val in df[df['isSameStrand']].iterrows():
        curr = df[(val['Chr'] == df['Chr']) & (df['isSameStrand'] == False)]
        if val['Strand'] == '-':
            curr = curr[(val['End'] - curr['Start'] < divergent_threshold) & 
                       (val['End'] - curr['Start'] > 0)]
        else: 
            curr = curr[(val['Start'] - curr['End'] < divergent_threshold) &
                       (val['Start'] - curr['End'] > 0 ) ]

        ## If there are divergent peaks, add this peak
        if len(curr) > 0: 
            inds_to_keep.append(ind)
    
    df = df.loc[inds_to_keep]
    return df



In [20]:
def tissue_specific_promoter(t,
                             anno_peaks,
                             peaks_tissue_merged,
                             txn_df,tss,
                             peak_count_thresh = 1, 
                             prom_bin=[-1000, 1000],
                             enhancer_d=40000,
                             divergent_thr=500,
                             outfolder='Results/peak_groups/', 
                             has_cs=True):

    ''' Function to create bed files relevant for a specific tissue
        Will save:
        a) All peaks that were found in tissue
        b) Peaks within prom_bin distance from reference & has divergent peak nearby (divergent_thr distance)
        c) Same as b, but no divergent restriction
        d) Same as c, but the peak center is now the annnotated start site instead of experimental
        e) ncRNA, possibly eRNA that are greater than enhancer_d from any annotated start sites.
        f) no_peaks: genes with no peaks nearby, even divergent peaks'''
    
    
    
    t_peaks = peaks_tissue_merged[peaks_tissue_merged.loc[:, t] > peak_count_thresh].index #Peaks in that tissue
    t_peaks = anno_peaks.loc[t_peaks]
    t_peaks = t_peaks[(~(t_peaks['Distance to TSS'] == np.infty))] ## Remove peaks that are on a scaffold with no gene

    ### Absence of peaks for that tissue.
    no_t_peaks = txn_df[txn_df['samples'].apply(lambda x: np.array(map(lambda y: t not in y,x)).any())]
    no_t_peaks = tss[tss['transcript_id'].apply(lambda x: x in no_t_peaks.index)]
    #no_t_peaks = peaks_tissue_merged[peaks_tissue_merged.loc[:, t] <= peak_count_thresh].index #Peaks not in the tissue
    #no_t_peaks = anno_peaks.loc[no_t_peaks]
    
    t_prom = t_peaks[(t_peaks['Distance to TSS'] > prom_bin[0])
                     & (t_peaks['Distance to TSS'] < prom_bin[1])]
    t_prom_isSame = t_prom[t_prom['isSameStrand']]
    
    
    t_enh = t_peaks[np.abs(t_peaks['Distance to TSS']) > enhancer_d]
    t_prom_div = filter_only_with_divergent(
        t_prom, divergent_threshold=divergent_thr)
    t_enh_div = filter_only_with_divergent(
        t_enh, divergent_threshold=divergent_thr)

    t_prom_div = t_prom_div.sort_values(['Distance to TSS'])
    
    #Look at RefSeq vs experimental TSS
    tss_withPeaks = t_prom_isSame['Nearest TSS'].unique()
    refSeq = tss[tss['transcript_id'].apply(lambda x: x in tss_withPeaks)] 
    # Dont want the whole gene length, just the start of the peak 
    refSeq['Start'] = refSeq['actual_start'] #actual_start takes care of +/- strand
    refSeq['End'] = refSeq['actual_start']
    
    ## Saving bed files
    out_dir = os.path.join(outfolder, t)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    
    save_peak_as_bed(
        t_peaks,
        os.path.join(out_dir, 'all_peaks'),
        name='index',
        score_col='Stat')
    
    save_peak_as_bed(
        t_prom_div,
        os.path.join(out_dir, 'confident_promoters'),
        name='index',
        score_col='Stat')
    
    save_peak_as_bed(
        t_prom_isSame,
        os.path.join(out_dir, 'confident_promoters_noDiv'),
        name='index',
        score_col='Stat')  
    
    
    save_peak_as_bed(
        t_enh_div,
        os.path.join(out_dir, 'confident_enhancers'),
        name='index',
        score_col='Stat')
    

    save_peak_as_bed(
        refSeq,
        os.path.join(out_dir, 'confident_promoters_noDiv_PICR'),
        name='index')
    save_peak_as_bed(
        no_t_peaks,
        os.path.join(out_dir, 'no_peaks'),
        name='index')

    return

In [17]:
tissues_with_atac = ['CHO','Brain','Liver','Lung','Kidney','Spleen','BMDMwt']#,'BMDM1hKLA']

In [19]:
anno_peaks.head()

,Chr,Start,End,Strand,Stat,Annotation,Nearest TSS,Distance to TSS,isSameStrand,Nearest gene
ID,,,,,,,,,,
0,NW_020822452.1,16590187,16590584,+,16.227333,promoter-TSS (ID=id181373;Parent=rna15526;Dbxr...,XM_027399668.1,-462.0,False,Clic4
1,NW_020822597.1,23595304,23595489,+,5.740000,Intergenic,XM_027420351.1,-23850.0,True,Baz2b
2,NW_020822680.1,728607,728829,+,7.579579,promoter-TSS (ID=id609942;Parent=rna53731;Dbxr...,XM_027432478.1,746.0,False,Irak1
3,NW_020822602.1,10883074,10883337,+,3.992000,Intergenic,XM_027423054.1,9079.0,True,Gbgt1
4,NW_020822601.1,37427816,37427999,+,129.031738,promoter-TSS (ID=id474643;Parent=rna41513;Dbxr...,XM_027421990.1,-11.0,True,Shf


In [21]:
%%time
for t in tissues_with_atac:
    print(t)
    tissue_specific_promoter(
        t=t,
        anno_peaks=anno_peaks,
        peaks_tissue_merged=peaks_tissue_merged,
        txn_df=txn_df,
        tss=tss, has_cs = False)

CHO


/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

KeyError: "['Peak Score'] not in index"

# TSS peaks near ATAC peaks. 
Using Bedtools

### Sort ATAC bed files

In [ ]:
## Sort ATAC bed files
for t in tissues_with_atac:
    print(t)
    atac_f = glob.glob(os.path.join(data_folder, t, 'ATAC/out/peak/macs2/overlap/optimal_set/*narrowPeak.gz'))[0]
    atac_f_out = atac_f.replace('.gz','') + '.sort'
    atac_f_out = os.path.join("Results/ATAC_results/",t,os.path.basename(atac_f).replace('.gz','.sort'))
    if not os.path.exists(os.path.join("Results/ATAC_results/",t)):
        os.mkdir(os.path.join("Results/ATAC_results/",t))
    
    cmd = 'bedtools sort -i %s > %s' % (atac_f,atac_f_out)   
    print(cmd)
    os.system(cmd)

### Get peaks close to TSS within distance of 500 bp

In [ ]:
def peaks_closest(f_a, f_b, out_f, dist_thresh):
    '''Function that calls bedtools closest to find the closest peak of two bed files, and awk to filter those within distance dist_thresh'''

    cmd = "bedtools closest -d -t first -a %s -b %s | awk '{if ($NF < %d) print }' > %s" % (
       f_a, f_b, dist_thresh,out_f) #t is to only report one of the atac peaks
    
#     cmd = "bedtools closest -d -t first -a %s -b %s  > %s" % (
#         f_a, f_b,out_f)
    
    print(cmd)
    os.system(cmd)
    return

In [ ]:
# Intersect ATAC and TSS
dist_thresh = 500 #500 basepairs left or right
outfolder = 'Results/peak_groups/'
f_names = [
    'all_peaks','confident_promoters', 'confident_enhancers', 'confident_promoters_noDiv',
    'confident_promoters_noDiv_PICR', 'no_peaks'
]


for t in tissues_with_atac:
    print(t)
    ## Outdirectory to save in
    out_dir = os.path.join(outfolder, t)
    
    #ATAC peak file
    atac_out = glob.glob(os.path.join("Results/ATAC_results", t , '*sort'))[0]
    
    ## The different TSS files
    for f in f_names:
        f_in = os.path.join(out_dir, f) + '.bed'
        out_f = '%s_atac_overlap_%dbp.bed' % (f_in,dist_thresh)
        peaks_closest(f_in,atac_out,out_f,dist_thresh=dist_thresh)

In [ ]:
ls Results/peak_groups/Brain/

In [ ]:
# Intersect ATAC and TSS
dist_thresh = 100 #500 basepairs left or right
outfolder = 'Results/peak_groups/'
f_names = [
    'all_peaks','confident_promoters', 'confident_enhancers', 'confident_promoters_noDiv',
    'confident_promoters_noDiv_PICR', 'no_peaks'
]


for t in tissues_with_atac:
    print(t)
    ## Outdirectory to save in
    out_dir = os.path.join(outfolder, t)
    
    #ATAC peak file
    atac_out = glob.glob(os.path.join("Results/ATAC_results", t , '*sort'))[0]
    
    ## The different TSS files
    for f in f_names:
        f_in = os.path.join(out_dir, f) + '.bed'
        out_f = '%s_atac_overlap_%dbp.bed' % (f_in,dist_thresh)
        peaks_closest(f_in,atac_out,out_f,dist_thresh=dist_thresh)

### Use wc -l to determine the number of peaks that are near ATAC, compared to total


In [ ]:
from subprocess import check_output

def wc(filename):
    return int(check_output(["wc", "-l", filename]).split()[0])


In [ ]:
dist_thresholds = [100,500]
atac_tss_df = pd.DataFrame()
for t in tissues_with_atac:
    print(t)
    out_dir = os.path.join(outfolder, t)
    ## The different TSS files
    for f in f_names:
        f_in = os.path.join(out_dir, f) + '.bed'
        
        for d in dist_thresholds:
            out_f = '%s_atac_overlap_%dbp.bed' % (f_in,d)
            print(f_in)
            pre_filt = wc(f_in)
            print(out_f)
            post_filt = wc(out_f)
            name = os.path.basename(f_in)
            frac = 1.0*post_filt/pre_filt
            atac_tss_df.at[t,name] = pre_filt
            name_atac = '_with_ATAC_%d' % d
            ## Add to dataframe
            
            atac_tss_df.at[t,name + name_atac] = post_filt
            atac_tss_df.at[t,name + name_atac + '_fraction'] = frac


In [ ]:
atac_tss_df

In [ ]:
col_names = np.array(atac_tss_df.columns.values)
#to_drop = col_names[map(lambda x: 'fraction' not in x or '100' not in x,atac_tss_df.columns.values)]
#output = atac_tss_df.drop(to_drop,axis=1) 
output = atac_tss_df.copy()
col_names = np.array(output.columns.values)
for i,name in enumerate((col_names)):
    name = name.replace('.bed','')
    #name = name.replace('fraction','')
    name = name.replace('ATAC','ATAC peak within')
    #name = 'Fraction of ' + name
    #name = name + 'bp'
    col_names[i] = name.replace('_',' ')
output.columns = col_names



In [ ]:
output.transpose()

---------

---------